# 제품 이상여부 판별 프로젝트


### 데이터 읽어오기


In [1]:
import pandas as pd

RANDOM_STATE = 110

train_data = pd.read_csv("C:/Users/KimDongyoung/Desktop/git_LGaimers5/Lg_aimers5/data/train_data_0817.csv")
test_data = pd.read_csv("C:/Users/KimDongyoung/Desktop/git_LGaimers5/Lg_aimers5/data/test_data_0817.csv")

In [2]:
# dam, fill1, fill2 공통 변수
var_dam_fill = [
    'Equipment_same_num',
    'PalletID_Collect_Result_encoded',
    'Production_Qty_Collect_Result',
    'WorkMode Collect Result'
]

In [3]:
# 전체 공통 변수
### correlation 확인을 위한 변수 리스트
var_all_corr = [
    'model_receip_encoded',
    'workorder_receip_encoded'
]

### train
var_all_train = [
    'target',
    'model_receip_encoded',
    'workorder_receip_encoded'
]

### test
var_all_test = [
    'Set ID',
    'target',
    'model_receip_encoded',
    'workorder_receip_encoded'
]

In [4]:
# '_Dam'을 포함하는 변수 선택
dam_variables = [var for var in train_data.columns if '_Dam' in var]

# train
final_columns_train = var_dam_fill + var_all_train + dam_variables
train_data_dam = train_data[final_columns_train]

# test 
final_columns_test = var_dam_fill + var_all_test + dam_variables
test_data_dam = test_data[final_columns_test]

In [5]:
# '_Fill1'을 포함하는 변수 선택
fill1_variables = [var for var in train_data.columns if '_Fill1' in var]

# train
final_columns_train = var_dam_fill + var_all_train + fill1_variables
train_data_fill1 = train_data[final_columns_train]

# test 
final_columns_test = var_dam_fill + var_all_test + fill1_variables
test_data_fill1 = test_data[final_columns_test]

In [6]:
# '_Fill2'을 포함하는 변수 선택
fill2_variables = [var for var in train_data.columns if '_Fill2' in var]

# train
final_columns_train = var_dam_fill + var_all_train + fill2_variables
train_data_fill2 = train_data[final_columns_train]

# test 
final_columns_test = var_dam_fill + var_all_test + fill2_variables
test_data_fill2 = test_data[final_columns_test]

In [7]:
# '_AutoClave'을 포함하는 변수 선택
autoclave_variables = [var for var in train_data.columns if '_AutoClave' in var]

# train
final_columns_train = var_all_train + autoclave_variables
train_data_autoclave = train_data[final_columns_train]

# test 
final_columns_test = var_all_test + autoclave_variables
test_data_autoclave = test_data[final_columns_test]

In [8]:
# 각 DataFrame의 칼럼 수 계산
num_columns_train_data = train_data.shape[1]
num_columns_train_data_dam = train_data_dam.shape[1]
num_columns_train_data_autoclave = train_data_autoclave.shape[1]
num_columns_train_data_fill1 = train_data_fill1.shape[1]
num_columns_train_data_fill2 = train_data_fill2.shape[1]

num_columns_test_data = test_data.shape[1]
num_columns_test_data_dam = test_data_dam.shape[1]
num_columns_test_data_autoclave = test_data_autoclave.shape[1]
num_columns_test_data_fill1 = test_data_fill1.shape[1]
num_columns_test_data_fill2 = test_data_fill2.shape[1]

# 각 DataFrame의 칼럼 수 출력
print("----train data-----")
print(f"train_data DataFrame의 칼럼 수: {num_columns_train_data}")
print(f"train_data_dam DataFrame의 칼럼 수: {num_columns_train_data_dam}")
print(f"train_data_autoclave DataFrame의 칼럼 수: {num_columns_train_data_autoclave}")
print(f"train_data_fill1 DataFrame의 칼럼 수: {num_columns_train_data_fill1}")
print(f"train_data_fill2 DataFrame의 칼럼 수: {num_columns_train_data_fill2}")
print("----test data-----")
print(f"test_data DataFrame의 칼럼 수: {num_columns_test_data}")
print(f"test_data_dam DataFrame의 칼럼 수: {num_columns_test_data_dam}")
print(f"test_data_autoclave DataFrame의 칼럼 수: {num_columns_test_data_autoclave}")
print(f"test_data_fill1 DataFrame의 칼럼 수: {num_columns_test_data_fill1}")
print(f"test_data_fill2 DataFrame의 칼럼 수: {num_columns_test_data_fill2}")

----train data-----
train_data DataFrame의 칼럼 수: 38
train_data_dam DataFrame의 칼럼 수: 21
train_data_autoclave DataFrame의 칼럼 수: 8
train_data_fill1 DataFrame의 칼럼 수: 13
train_data_fill2 DataFrame의 칼럼 수: 13
----test data-----
test_data DataFrame의 칼럼 수: 39
test_data_dam DataFrame의 칼럼 수: 22
test_data_autoclave DataFrame의 칼럼 수: 9
test_data_fill1 DataFrame의 칼럼 수: 14
test_data_fill2 DataFrame의 칼럼 수: 14


---

## Optuna

스레스홀드 0.3으로 맞춘상태에서 튜닝 진행한 것

In [9]:
import optuna
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

# 'Normal'과 'AbNormal'을 숫자로 변환
train_data_autoclave['target'] = train_data_autoclave['target'].map({'Normal': 0, 'AbNormal': 1})

# 스레드홀드 설정
THRESHOLD = 0.3


def objectiveXGB(trial, x_tr, y_tr, x_val, y_val):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 1200, 3200)
        , 'learning_rate': trial.suggest_float('learning_rate', 0.002, 0.2)
        , 'max_depth': trial.suggest_int('max_depth', 3, 10)

        , 'alpha': trial.suggest_float('alpha', 0.00001, 0.01, log=True)
        , 'gamma': trial.suggest_float('gamma', 0.00001, 0.01, log=True)

        , 'reg_alpha' : trial.suggest_float('reg_alpha', 0.01, 1)
        , 'reg_lambda' : trial.suggest_float('reg_lambda', 0.01, 1)
        
        , 'colsample_bytree' : trial.suggest_float('colsample_bytree', 0.01, 1)
        , 'subsample' : trial.suggest_float('subsample', 0.03, 1)
        , 'objective': 'binary:logistic'  # 이진 분류
        , 'tree_method' : "exact"        
        , 'random_state': RANDOM_STATE
    }
       
    model = XGBClassifier(**param)
    model.fit(x_tr, y_tr)
    pred_proba = model.predict_proba(x_val)[:, 1]  # 양성 클래스 확률
    pred = (pred_proba >= THRESHOLD).astype(int)  # 스레드홀드에 따른 예측
    
    score = f1_score(y_val, pred, average="binary")
    
    return score

# 데이터셋 분할
x_train, x_val, y_train, y_val = train_test_split(
    train_data_autoclave.drop("target", axis=1),
    train_data_autoclave["target"],
    test_size=0.2,
    shuffle=True,
    random_state=RANDOM_STATE,
)

# 하이퍼 파라미터 튜닝
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=RANDOM_STATE))
study.optimize(lambda trial: objectiveXGB(trial, x_train, y_train, x_val, y_val), n_trials=400)

print('Best trial: score {}, \nparams {}'.format(study.best_trial.value, study.best_trial.params))



C:\Users\KimDongyoung\AppData\Local\Temp\ipykernel_6928\2375461130.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data_autoclave['target'] = train_data_autoclave['target'].map({'Normal': 0, 'AbNormal': 1})
[I 2024-08-19 15:00:56,203] A new study created in memory with name: no-name-6b2581f4-bb9f-450a-97ec-163b0e365f9f
[I 2024-08-19 15:01:04,328] Trial 0 finished with value: 0.21243523316062177 and parameters: {'n_estimators': 1432, 'learning_rate': 0.13237416558573836, 'max_depth': 6, 'alpha': 0.000689203929104206, 'gamma': 0.0010926263191103419, 'reg_alpha': 0.685638497930364, 'reg_lambda': 0.9554598960277283, 'colsample_bytree': 0.640850709997791, 'subsample': 0.5275552995141115}. Best is trial 0 with value: 0.21243523316062177.
[I 2

Best trial: score 0.25604551920341395, 
params {'n_estimators': 1707, 'learning_rate': 0.0321470219836192, 'max_depth': 7, 'alpha': 7.368872823521818e-05, 'gamma': 0.0007930035188326916, 'reg_alpha': 0.644199314174124, 'reg_lambda': 0.588270569327407, 'colsample_bytree': 0.883929103208459, 'subsample': 0.2534703342501092}


Best trial: score 0.25604551920341395

params {'n_estimators': 1707, 'learning_rate': 0.0321470219836192, 'max_depth': 7, 'alpha': 7.368872823521818e-05, 'gamma': 0.0007930035188326916, 'reg_alpha': 0.644199314174124, 'reg_lambda': 0.588270569327407, 'colsample_bytree': 0.883929103208459, 'subsample': 0.2534703342501092}

.